In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
import cartopy.crs as ccrs
import cmocean
from cartopy.feature import LAND

In [ ]:
from eofs.xarray import Eof

In [ ]:
import sys
sys.path.append("scripts/")
import processing as proc
import spectrumfct as spectrum
import statsfct as stats

In [ ]:
rho0 = 1026 # Reference density [kg/m^3]

In [ ]:
ref_lat = 40 # Reference latitude for AMOC calculations (°N)

In [ ]:
lowpass_cutoff = 70

## Imports

### AMOC timeseries

In [ ]:
model_list = pd.read_csv("model_list.csv").set_index("Model")
sel_models = list(model_list.index)

In [ ]:
# Import streamfunction fields
amoc_strf = {}
amoc_strf_detr = {}
amoc_40n_detr = {}
for model in sel_models:
    member = model_list.loc[model,"Member"]
    amoc_files = glob.glob("/home/omehling/work/cmip6/piControl_processed/amoc/CMIP6_{}_*{}*.nc".format(model, member))
    if len(amoc_files) != 1:
        raise ValueError("{} files found for {}".format(len(amoc_files), model))
    ds_imp = xr.open_dataset(amoc_files[0], use_cftime=True)
    # Rename "rlat" to "lat" if needed
    if 'rlat' in list(ds_imp.dims):
        ds_imp = ds_imp.rename({'rlat':'lat'})
    # Select streamfunction variable
    moc_var_name = "msftmz" if "msftmz" in list(ds_imp.data_vars) else "msftyz"
    amoc_strf[model] = ds_imp[moc_var_name].drop("sector")/rho0*1e-6 # Units: kg/s -> Sv
    amoc_strf_detr[model] = proc.detrend_xr(amoc_strf[model], 2) # Quadratic detrending
    amoc_40n_detr[model] = amoc_strf_detr[model].sel(lat=ref_lat, method="nearest").sel(lev=slice(500,None)).max("lev")
    
    print(model)

### FW content & SSH fields

In [ ]:
def load_cell_area(model_code):
    model_grid = "gn"
    if model_code == "CESM2": model_grid = "gr"
    
    areacello_path = glob.glob("/home/omehling/synda-CMIP/*/*/piControl/*/Ofx/areacello/{}/*/areacello*{}*.nc".format(model_grid,model_code.replace('_','*')))
    if len(areacello_path)==0:
        areacello_path = glob.glob("/home/omehling/synda-CMIP/*/*/historical/*/Ofx/areacello/{}/*/areacello*{}*.nc".format(model_grid,model_code.replace('_','*')))
    return xr.open_dataset(areacello_path[0])["areacello"]

In [ ]:
fwc = {}
fwc_detr = {}
areacello = {}

for i, model_id in enumerate(sel_models):
    member = model_list.loc[model_id,"Member"]
    filename_surf = glob.glob("/work/users/omehling/cmip6/piControl_salinity/fwc_ao/fwc_Omon_{}_*{}*.nc".format(model_id, member))
    if len(filename_surf) != 1:
        print("Warning: Skipping {}, {} files found instead of 1".format(model_id, len(filename_surf)))
        continue
    ds_load = xr.open_dataset(filename_surf[0], use_cftime=True).rename({'so':'fwc'})
    fwc[model_id] = ds_load['fwc']
    fwc_detr[model_id] = proc.detrend_xr(fwc[model_id], 2, keep_mean=True)
    
    areacello[model_id] = load_cell_area(model_id)
    print(model_id)

In [ ]:
zos_raw = {}
zos_detr = {}

for i, model_id in enumerate(sel_models):
    member = model_list.loc[model_id,"Member"]
    filname_zos = glob.glob("/home/omehling/work/cmip6/piControl_processed/zos/CMIP6_{}*{}*r360x180*.nc".format(model_id, member))
    if len(filname_zos) != 1:
        print("Warning: Skipping {}, {} files found instead of 1".format(model_id, len(filname_zos)))
        continue
    zos_load = xr.open_dataset(filname_zos[0], use_cftime=True)["zos"]
    zos_raw[model_id] = zos_load
    zos_detr[model_id] = proc.detrend_xr(zos_load, 2, keep_mean=True)
    
    print(model_id)

### Freshwater transport timeseries

In [ ]:
fwt_path = "/home/omehling/work/cm6-amoc/calc-fwt/"

def load_fwt(model_id):
    model_code = model_list.loc[model_id,"Abbrev"]
    filenames_fwt = sorted(glob.glob(fwt_path+"fwt_"+model_code+"/fwt_*.csv"))
    df_fwts = []
    for fn in filenames_fwt:
        df_fwts.append(pd.read_csv(fn))
    df_fwts = pd.concat(df_fwts, axis=0).sort_values("time")
    start_date = df_fwts["time"].iloc[0].split("-")
    start_date = "{}-{}".format(start_date[0], start_date[1])
    df_fwts["time"] = xr.cftime_range(start=start_date, periods=len(df_fwts), freq="M")
    df_fwts = df_fwts.set_index("time")
    
    return df_fwts

In [ ]:
fwt_liq = {}
fwt_liq_ann_detr = {}
for model in sel_models:
    fwt_liq[model] = load_fwt(model)
    fwt_liq_ann_detr[model] = proc.detrend_xr(fwt_liq[model].to_xarray().groupby("time.year").mean("time"), 2, time_dim="year")
    print(model)

## Processing

### FW content EOFs

In [ ]:
def ao_mask(model_code, include_baffin=False):
    areacello_model = load_cell_area(model_code)
    # Determine model grid properties
    lat_name, lon_name = proc.lat_lon_name(areacello_model)
    lon360 = (areacello_model[lon_name].max().item()>190)
    
    ao_east = np.array([[0., 80.], [25, 80], [25, 68], [40,61], [180,66.5], [180,90], [0,90]])
    if include_baffin:
        # Includes Baffin Bay up to Davis Strait
        ao_west = np.array([[-180,90], [-180,66.5], [-160,66.5], [-87,66.5], [-85,70], [-40,70], [-30., 80.], [0,80], [0,90]])
    else:
        # Otherwise, delimit Arctic Ocean approximately by the CAA gateways
        ao_west = np.array([[-180,90], [-180,66.5], [-160,66.5], [-87,66.5], [-80,70], [-80,77], [-75,80], [-40,80], [-40,70], [-30., 80.], [0,80], [0,90]])
    ao_reg = regionmask.Regions([ao_east, ao_west], names=["West", "East"], abbrevs=["AOw", "AOe"])
    ao_regmask = ao_reg.mask(areacello_model, lon_name=lon_name, lat_name=lat_name, wrap_lon=lon360)
    
    return ~ao_regmask.isnull()

In [ ]:
fwc_eofs = {}
fwc_pcs = {}
fwc_pc_eigenvalues = {}
fwc_pc_variance = {}
fwc_total_detr = {}

for model_id in sel_models:
    # Select and center FWC
    areacello_ao = areacello[model_id].where(ao_mask(model_id),drop=True)
    space_dims = list(set(fwc_detr[model_id].dims)-set(["time"]))
    fwc_ao = fwc_detr[model_id].where(ao_mask(model_id),drop=True).transpose("time", *list(areacello_ao.dims))
    fwc_total_detr[model_id] = (fwc_ao*areacello_ao).sum(space_dims) # non-centered data needed for total FWC
    fwc_total_detr[model_id] = fwc_total_detr[model_id].rename({"time": "year"})*1e-9
    fwc_total_detr[model_id]["year"] = [t.year for t in fwc_total_detr[model_id].year.values]
    
    # Compute & save EOFs
    if model_id == "CESM2":
        fwc_eof_solver = Eof(fwc_ao, weights=np.sqrt(areacello_ao.fillna(0)))
    else:
        fwc_eof_solver = Eof(fwc_ao.drop(proc.lat_lon_name(areacello[model_id])), weights=np.sqrt(areacello_ao.drop(proc.lat_lon_name(areacello[model_id])).fillna(0)))
    eof_signs = np.sign(fwc_eof_solver.eofs(neofs=10).weighted(np.sqrt(areacello_ao.fillna(0))).mean(list(fwc_eof_solver.eofs(neofs=1).sel(mode=0).dims)))
    fwc_eofs[model_id] = fwc_eof_solver.eofs(neofs=10)*eof_signs
    fwc_pcs[model_id] = fwc_eof_solver.pcs(npcs=10, pcscaling=2).rename({"time": "year"})*eof_signs*1e-9
    fwc_pcs[model_id]["year"] = [t.year for t in fwc_pcs[model_id].year.values]
    fwc_pc_eigenvalues[model_id] = fwc_eof_solver.eigenvalues(neigs=10)
    fwc_pc_variance[model_id] = fwc_eof_solver.varianceFraction(neigs=10)
    
    print(model_id)

### FW content regression onto AMOC

In [ ]:
amoc_40n_filt = {}
fwc_filt = {}
zos_filt = {}

for i, model_id in enumerate(sel_models):
    # Select and filter
    amoc_sel, fwc_sel = xr.align(proc.time2year(amoc_40n_detr[model_id].drop("lat")), proc.time2year(fwc_detr[model_id]), join="inner")
    amoc_sel, zos_sel = xr.align(amoc_sel, proc.time2year(zos_detr[model_id]), join="inner")
    amoc_40n_filt[model_id] = proc.filter_xr(amoc_sel, lowpass_cutoff, time_dim="year", detrend=False)
    fwc_filt[model_id] = proc.filter_xr(fwc_sel, lowpass_cutoff, time_dim="year", detrend=False)
    zos_filt[model_id] = proc.filter_xr(zos_sel, lowpass_cutoff, time_dim="year", detrend=False)
    
    print(model_id)

In [ ]:
lagreg_amoc_zos = {}
lagreg_amoc_fwc = {}
maxlag_amoc_fwc = {}

for i, model_id in enumerate(sel_models):
    # AMOC-EOF1 correlation (same as below, but only to find max. lag)
    amoc_sel_std = ((amoc_40n_detr[model_id]-amoc_40n_detr[model_id].mean())/amoc_40n_detr[model_id].std()).swap_dims({"time": "year"})
    fwc_sel_std = (fwc_pcs[model_id].sel(mode=0)-fwc_pcs[model_id].sel(mode=0).mean())/fwc_pcs[model_id].sel(mode=0).std()
    amoc_sel_std = amoc_sel_std.sel(year=fwc_sel_std["year"]) # align time axes

    lagcorr_amoc_fwc = stats.lagregs(amoc_sel_std, fwc_sel_std, np.arange(-100,101), "year")
    maxlag_amoc_fwc[model_id] = np.arange(-30,31)[lagcorr_amoc_fwc.sel(lag=slice(-30,30)).argmax().item()]
    
    # Calculate instantaneous regression + lagged regression at max. correlation
    sel_lags = [0, maxlag_amoc_fwc[model_id]] if maxlag_amoc_fwc[model_id]!=0 else [0]
    lagreg_amoc_fwc[model_id] = stats.lagregs(amoc_40n_filt[model_id], fwc_filt[model_id], sel_lags, "year")
    lagreg_amoc_zos[model_id] = stats.lagregs(amoc_40n_filt[model_id], zos_filt[model_id], sel_lags, "year")

## Plots

### Arctic Ocean map

(Suppl. Fig. S1)

In [ ]:
import regionmask

In [ ]:
# Load bathymetry
bathy = xr.open_dataset("/home/omehling/data/bathymetry/GEBCO_2023_01deg.nc")["elevation"]
bathy = proc.ds_to_180(bathy.where(bathy<=0))

In [ ]:
# Arctic Ocean mask
ao_east = np.array([[0., 80.], [25, 80], [25, 68], [40,61], [180,66.5], [180,90], [0,90]])
ao_west = np.array([[-180,90], [-180,66.5], [-160,66.5], [-87,66.5], [-85,70], [-40,70], [-30., 80.], [0,80], [0,90]])
ao_reg = regionmask.Regions([ao_east, ao_west], names=["West", "East"], abbrevs=["AOw", "AOe"])
ao_regmask = ao_reg.mask(bathy, lon_name="lon", lat_name="lat", wrap_lon=False)

In [ ]:
ao_mask_bathy = np.logical_and(ao_regmask>=0, bathy<0)
ao_invmask_bathy = np.logical_and(ao_regmask.isnull(), bathy<0)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6), subplot_kw={"projection": ccrs.NorthPolarStereo()})
ax.set_rasterized(True)
c = bathy.sel(lat=slice(50,None)).plot.contourf(
    levels=[-3000,-2000,-1250,-1000,-650,-200,0], colors=sns.color_palette("Blues_r",8,desat=0.8),
    ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False
)
c2 = ao_invmask_bathy.where(ao_invmask_bathy==True).sel(lat=slice(50,None)).plot.contourf(
    levels=[0.5,2], colors="C7", alpha=0.4,
    ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False
)
ax.set_extent([-180, 180, 63, 90], ccrs.PlateCarree())
ax.gridlines(lw=.2, color="k", alpha=.5)
ax.coastlines(lw=.4, color="w")

ax.add_feature(LAND, facecolor='#777777', zorder=2.5)
ax.spines['geo'].set_linewidth(0)

cb = fig.colorbar(c, ax=ax, orientation='horizontal', shrink=0.6, aspect=25,
                extend='min', label='Bathymetry [m]')
cb.ax.tick_params(labelsize=8)

fig.savefig("figures/Map_Arctic_Ocean.pdf", bbox_inches="tight", dpi=300)

### Freshwater content EOFs

(Suppl. Fig. S6)

In [ ]:
fig,axes=plt.subplots(3,3,figsize=(10,10),subplot_kw=dict(projection=ccrs.NorthPolarStereo()))
fig.subplots_adjust(wspace=0.05)
for i, model_id in enumerate(sel_models):
    lat_name, lon_name = proc.lat_lon_name(areacello[model_id])
    
    lr_eof = stats.linreg_np(
        (fwc_pcs[model_id].sel(mode=0)/fwc_pcs[model_id].sel(mode=0).std()).values,
        fwc_detr[model_id].transpose(*(["time"]+list(areacello[model_id].dims))).values
    )
    
    omask_orig = ~fwc_detr[model_id].isel(time=0,drop=True).isnull()
    areacello_ao = areacello[model_id].where(ao_mask(model_id))
    
    ax=axes.flat[i]
    c = ax.pcolormesh(areacello_ao[lon_name], areacello_ao[lat_name],
                      lr_eof,
                    transform=ccrs.PlateCarree(), cmap="cmo.balance_r", rasterized=True, vmin=-3, vmax=3)
    ax.coastlines(lw=.4, color="C7")
    ax.set_extent([-180, 180, 62, 90], ccrs.PlateCarree())
    ax.set_title(model_id+" ({:.0f} %)".format(fwc_pc_variance[model_id].sel(mode=0)*100), fontsize=11)
    ax.gridlines(lw=.5)
    
    print(model_id)
    
cbar_ax = fig.add_axes([0.93, 0.3, 0.015, 0.4])
cbar = fig.colorbar(c, cax=cbar_ax, extend='max', ticks=np.arange(-3,4,1))
cbar.set_label('Freshwater content regressed onto PC1 [m]', fontsize=9)

fig.savefig('figures/Suppl_fwc-EOFs.pdf',bbox_inches='tight',dpi=300)
fig.savefig('figures/png/Suppl_fwc-EOFs.png',bbox_inches='tight',dpi=300)

### FW content regression onto AMOC (Fig. 2)

(+ Suppl. Fig. S5)

In [ ]:
plot_maxlag = False

layout_grid = [['Map1', 'Map2', 'Map3'],
               ['Map1', 'Map2', 'Map3'],
               ['Line1', 'Line2', 'Line3'],
               ['Map4', 'Map5', 'Map6'],
               ['Map4', 'Map5', 'Map6'],
               ['Line4', 'Line5', 'Line6'],
               ['Map7', 'Map8', 'Map9'],
               ['Map7', 'Map8', 'Map9'],
               ['Line7', 'Line8', 'Line9']
              ]
map_labels = tuple(["Map{}".format(i) for i in range(1,10)])
line_labels = tuple(["Line{}".format(i) for i in range(1,10)])
fig, axes = plt.subplot_mosaic(layout_grid, figsize=(7.2,11), layout="constrained",
                              per_subplot_kw={map_labels: {"projection": ccrs.NorthPolarStereo()}}
                              )

for i, model_id in enumerate(sel_models):
    # Lagreg plots (AMOC vs freshwater PC1)
    line_ax = axes["Line{}".format(i+1)]

    amoc_sel_std = ((amoc_40n_detr[model_id]-amoc_40n_detr[model_id].mean())/amoc_40n_detr[model_id].std()).swap_dims({"time": "year"})
    fwc_sel_std = (fwc_pcs[model_id].sel(mode=0)-fwc_pcs[model_id].sel(mode=0).mean())/fwc_pcs[model_id].sel(mode=0).std()
    amoc_sel_std = amoc_sel_std.sel(year=fwc_sel_std["year"]) # align time axes

    lagcorr_amoc_fwc = stats.lagregs(amoc_sel_std, fwc_sel_std, np.arange(-100,101), "year")
    lagcorr_amoc_fwc.plot(
        ax=line_ax, lw=1.6, c="#2843a2", label="FW content PC1"
    )

    lower_conf, upper_conf = stats.confidence_levels_lagreg(
        amoc_sel_std, fwc_sel_std, np.arange(-100,101), ci_level=0.95, time_dim="year"
    )
    #line_ax.axhline(lower_conf, lw=1, c="#2843a2", ls="--", dashes=(3, 2)) # do not plot lower conf for one-sided test
    line_ax.axhline(upper_conf, lw=1, c="#2843a2", ls="--", dashes=(3, 2))
    if plot_maxlag:
        line_ax.axvline(maxlag_amoc_fwc[model_id], lw=1.2, c="C7", ls="--", dashes=(3, 2))
    
    # Style lagreg axes
    line_ax.set(ylim=(-0.75,0.75), yticks=np.arange(-0.6,0.9,0.3), xlim=(-100,100), xticks=np.arange(-80,120,40))
    if i<6: line_ax.set_xticklabels([])
    if i%3!=0: line_ax.set_yticklabels([])
    
    line_ax.axhline(0, c="k", lw=.4)
    line_ax.axvline(0, c="k", lw=.4)
    line_ax.set_xlabel(""); line_ax.set_xlabel("")
    line_ax.set_title("")
    
    line_ax.spines['right'].set_visible(False)
    line_ax.spines['top'].set_visible(False)


    # Map plots (freshwater regression)
    ax = axes["Map{}".format(i+1)]

    lat_name, lon_name = proc.lat_lon_name(areacello[model_id])

    omask_orig = ~fwc_detr[model_id].isel(time=0,drop=True).isnull()
    omask_regrid = ~zos_detr[model_id].isel(time=0,drop=True).isnull()
    areacello_ao = areacello[model_id].where(ao_mask(model_id))
    if plot_maxlag:
        plot_lag = maxlag_amoc_fwc[model_id]
    else:
        plot_lag = 0

    c = ax.pcolormesh(
        fwc_detr[model_id][lon_name], fwc_detr[model_id][lat_name],
        lagreg_amoc_fwc[model_id].sel(lag=plot_lag).where(omask_orig),
        transform=ccrs.PlateCarree(), cmap="cmo.balance_r", vmin=-2.5, vmax=2.5, rasterized=True
    )
    cs = ax.contour(
        zos_detr[model_id]["lon"], zos_detr[model_id]["lat"],
        lagreg_amoc_zos[model_id].sel(lag=plot_lag).where(omask_regrid)*100,
        transform=ccrs.PlateCarree(), colors="k", levels=np.arange(-10,16,2), linewidths=0.5
    )
    #plt.clabel(cs, np.arange(-.1,.15,.02), fontsize=6.5, inline_spacing=1) # inline_spacing=1
    ax.clabel(cs,  colors=['black'], manual=False, inline=False, fontsize=7)

    
    # Style maps
    ax.set_extent([-180, 180, 63, 90], ccrs.PlateCarree())
    ax.gridlines(lw=.2, color="k", alpha=.5)
    ax.coastlines(lw=.4, color="C7")
    ax.set_title(model_id, fontsize=11)
    
    print(model_id, maxlag_amoc_fwc[model_id])

cbar_ax = fig.add_axes([1.03, 0.3, 0.015, 0.4])
cbar = fig.colorbar(c, cax=cbar_ax, extend='both', ticks=np.arange(-3,4,1))
cbar.set_label('Freshwater content regressed onto AMOC [m/Sv]', fontsize=9)

fig.supylabel('AMOC–FWC PC1 correlation coefficient',x=-0.02,y=0.4,size=10)
fig.supxlabel('Lag [years]',size=10,y=-0.015,x=0.53);

if plot_maxlag:
    fig.savefig("figures/Suppl_Freshwater-lagreg-maxlag.pdf", bbox_inches="tight", dpi=300)
    fig.savefig("figures/png/Suppl_Freshwater-lagreg-maxlag.png", bbox_inches="tight", dpi=250)
else:
    fig.savefig("figures/Freshwater-lagreg.pdf", bbox_inches="tight", dpi=300)
    fig.savefig("figures/png/Freshwater-lagreg.png", bbox_inches="tight", dpi=250)

### Freshwater transport across Fram Strait

(Suppl. Fig. S7)

In [ ]:
fig, ax = plt.subplots()

#cpal = sns.color_palette("husl", len(sel_models))

sel_models_sep = ['CanESM5', 'EC-Earth3', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'UKESM1-0-LL', 'ACCESS-ESM1-5', 'CESM2', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-LR']
col_nemo = sns.color_palette("ch:s=.25,rot=-.35", 12)[5] # "C0"
col_other = sns.color_palette("ch:s=.15,rot=.45", 12)[4] # "C5"
#col_models = [col_other, col_nemo, col_other, col_nemo, col_nemo, col_nemo, col_other, col_other, col_nemo]
#ls_models = ["-", "-", "--", "--", "-.", (0, (0.8, 0.8)), "-.", (0, (0.8, 0.8)), (0, (6,2))]
ls_models = ["-", "--", "-.", (0, (0.8,0.8)), (0, (6,2))]
lagregs_amoc_fwt_nemo = []
lagregs_amoc_fwt_other = []

ax.axvline(0, c="k", lw=.7)
ax.axhline(0, c="k", lw=.7)
for i, model_id in enumerate(sel_models_sep):
    amoc_filtered = proc.filter_xr(amoc_40n_detr[model_id].swap_dims({"time": "year"}).drop("time"), lowpass_cutoff, "year", detrend=False)
    fwt_fram_filtered = proc.filter_xr(fwt_liq_ann_detr[model_id]["fwt_fram"], lowpass_cutoff, "year", detrend=False)
    
    lagreg_amoc_fwt = stats.lagregs(
        amoc_filtered,
        fwt_fram_filtered,
        np.arange(-100,101), "year"
    )
    
    if i<5:
        # NEMO models
        col_sel = col_nemo
        ls_sel = ls_models[i]
        lagregs_amoc_fwt_nemo.append(lagreg_amoc_fwt)
    else:
        # non-NEMO models
        col_sel = col_other
        ls_sel = ls_models[i-5]
        lagregs_amoc_fwt_other.append(lagreg_amoc_fwt)
    
    amoc_aligned, fwt_aligned = xr.align(amoc_filtered,fwt_fram_filtered, join="inner")
    lower_conf, upper_conf = stats.confidence_levels_lagreg(
        amoc_aligned, fwt_aligned, np.arange(-100,101), ci_level=0.95, time_dim="year"
    )
    
    lagreg_amoc_fwt.where(np.logical_or(lagreg_amoc_fwt>upper_conf, lagreg_amoc_fwt<lower_conf)).plot(lw=2.1, c=col_sel, ls=ls_sel)
    lagreg_amoc_fwt.plot(label=model_id, lw=1.2, c=col_sel, ls=ls_sel)
    if i==4:
        xr.concat(lagregs_amoc_fwt_nemo, dim="model").mean("model").plot(c=sns.color_palette("ch:s=.25,rot=-.35", 12)[9], lw=4, label="Mean (NEMO models)")
    print(model_id, np.arange(-100,101)[lagreg_amoc_fwt.argmax().item()])
    
xr.concat(lagregs_amoc_fwt_other, dim="model").mean("model").plot(c=sns.color_palette("ch:s=.15,rot=.45", 12)[7], lw=4, label="Mean (non-NEMO models)")

fig.legend(ncols=2, loc="upper center", bbox_to_anchor=(0.5,0), frameon=False)
ax.set(
    title="", xlim=(-100,100), ylim=(-450,450), xticks=np.arange(-100,110,20),
    ylabel="Freshwater transport regression\nonto AMOC [km$^3$ yr$^{-1}$/Sv]", xlabel="Lag [years]"
)
sns.despine()

fig.savefig("figures/Suppl_FW_transport_Fram.pdf", bbox_inches="tight")